In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings # Ignores any warning
warnings.filterwarnings("ignore")

train = pd.read_csv("./data/train_prepro.csv") 
test = pd.read_csv("./data/test_prepro.csv") 

In [2]:
train.head()

,Unnamed: 0,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Small,Tier 3,Grocery Store,732.3800
4,4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
test.head()

,Unnamed: 0,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,Small,Tier 2,Supermarket Type1
2,2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,Small,Tier 3,Grocery Store
3,3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,Small,Tier 2,Supermarket Type1
4,4,FDY38,13.600,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [4]:
train = train.drop(['Unnamed: 0'], axis = 1)
test = test.drop(['Unnamed: 0'], axis = 1)

In [7]:
train.groupby(by=['Item_Identifier', 'Item_Visibility']).count()

Item_Weight  Item_Fat_Content  Item_Type  \
Item_Identifier Item_Visibility                                             
DRA12           0.000000                   2                 2          2   
                0.040912                   1                 1          1   
                0.041113                   1                 1          1   
                0.041178                   1                 1          1   
                0.068535                   1                 1          1   
...                                      ...               ...        ...   
NCZ54           0.083359                   1                 1          1   
                0.083489                   1                 1          1   
                0.083528                   1                 1          1   
                0.083699                   1                 1          1   
                0.145952                   1                 1          1   

                                 Item_MRP  Outlet_Identifier  \
Item_Identifier Item_Visibility                                
DRA12           0.000000                2                  2   
                0.040912                1                  1   
                0.041113                1                  1   
                0.041178                1                  1   
                0.068535                1                  1   
...                                   ...                ...   
NCZ54           0.083359                1                  1   
                0.083489                1                  1   
                0.083528                1                  1   
                0.083699                1                  1   
                0.145952                1                  1   

                                 Outlet_Establishment_Year  Outlet_Size  \
Item_Identifier Item_Visibility                                           
DRA12           0.000000                                 2            2   
                0.040912                                 1            1   
                0.041113                                 1            1   
                0.041178                                 1            1   
                0.068535                                 1            1   
...                                                    ...          ...   
NCZ54           0.083359                                 1            1   
                0.083489                                 1            1   
                0.083528                                 1            1   
                0.083699                                 1            1   
                0.145952                                 1            1   

                                 Outlet_Location_Type  Outlet_Type  \
Item_Identifier Item_Visibility                                      
DRA12           0.000000                            2            2   
                0.040912                            1            1   
                0.041113                            1            1   
                0.041178                            1            1   
                0.068535                            1            1   
...                                               ...          ...   
NCZ54           0.083359                            1            1   
                0.083489                            1            1   
                0.083528                            1            1   
                0.083699                            1            1   
                0.145952                            1            1   

                                 Item_Outlet_Sales  
Item_Identifier Item_Visibility                     
DRA12           0.000000                         2  
                0.040912                         1  
                0.041113                         1  
                0.041178                      

In [9]:
len(train[train['Item_Visibility']==0])

526

In [15]:
visibility_item_avg = train.pivot_table(values='Item_Visibility',index='Item_Identifier')
print(visibility_item_avg)

                 Item_Visibility
Item_Identifier                 
DRA12                   0.031956
DRA24                   0.048062
DRA59                   0.134718
DRB01                   0.082126
DRB13                   0.008002
...                          ...
NCZ30                   0.024956
NCZ41                   0.051623
NCZ42                   0.009044
NCZ53                   0.027775
NCZ54                   0.080426

[1559 rows x 1 columns]


In [16]:
def impute_visibility_mean(cols):
    visibility = cols[0]
    item = cols[1]
    if visibility == 0:
        return visibility_item_avg['Item_Visibility'][visibility_item_avg.index == item]
    else:
        return visibility

In [18]:
print ('Orignal #missing - train : {} | test : {}'.format(sum(train['Item_Visibility']==0), 
                                                          sum(test['Item_Visibility']==0)))
train['Item_Visibility'] = train[['Item_Visibility','Item_Identifier']].apply(impute_visibility_mean,axis=1).astype(float)
test['Item_Visibility'] = test[['Item_Visibility','Item_Identifier']].apply(impute_visibility_mean,axis=1).astype(float)
print ('Final #missing - train : {} | test : {}'.format(sum(train['Item_Visibility']==0),
                                                        sum(test['Item_Visibility']==0)))

Orignal #missing - train : 526 | test : 353
Final #missing - train : 0 | test : 0


In [20]:
train['Outlet_Years'] = 2013 - train['Outlet_Establishment_Year']
test['Outlet_Years'] = 2013 - test['Outlet_Establishment_Year']

In [22]:
train['Outlet_Years'].describe()

count    8523.000000
mean       15.168133
std         8.371760
min         4.000000
25%         9.000000
50%        14.000000
75%        26.000000
max        28.000000
Name: Outlet_Years, dtype: float64

In [23]:
test['Outlet_Years'].describe()

count    5681.000000
mean       15.171097
std         8.372256
min         4.000000
25%         9.000000
50%        14.000000
75%        26.000000
max        28.000000
Name: Outlet_Years, dtype: float64

In [24]:
train['Item_Type_Combined'] = train['Item_Identifier'].apply(lambda x: x[0:2])

In [25]:
train['Item_Type_Combined'].value_counts()

FD    6125
NC    1599
DR     799
Name: Item_Type_Combined, dtype: int64

In [26]:
test['Item_Type_Combined'] = test['Item_Identifier'].apply(lambda x: x[0:2])
test['Item_Type_Combined'].value_counts()

FD    4076
NC    1087
DR     518
Name: Item_Type_Combined, dtype: int64

In [27]:
# Item_Fat_Content modification - LF : Low Fat, reg : Regular, low fat: Low Fat

train['Item_Fat_Content'] = train['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})

print(train['Item_Fat_Content'].value_counts())

Low Fat    5517
Regular    3006
Name: Item_Fat_Content, dtype: int64


In [33]:
test['Item_Fat_Content'] = test['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})

print(test['Item_Fat_Content'].value_counts())

Low Fat    3668
Regular    2013
Name: Item_Fat_Content, dtype: int64


In [34]:
# NC(Non-Consumables) -> Non-Edible
train.loc[train['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
print(train['Item_Fat_Content'].value_counts())

test.loc[train['Item_Type_Combined']=="Non-Consumable",'Item_Fat_Content'] = "Non-Edible"
print(test['Item_Fat_Content'].value_counts())

Low Fat    3668
Regular    2013
Name: Item_Fat_Content, dtype: int64

In [35]:
func = lambda x: x['Item_Visibility']/visibility_item_avg['Item_Visibility'][visibility_item_avg.index == x['Item_Identifier']][0]


In [38]:
train['Item_Visibility_MeanRatio'] = train.apply(func,axis=1).astype(float)
test['Item_Visibility_MeanRatio'] = test.apply(func,axis=1).astype(float)

In [39]:
print(train['Item_Visibility_MeanRatio'].describe())
print(test['Item_Visibility_MeanRatio'].describe())

count    8523.000000
mean        1.061715
std         0.237149
min         0.678833
25%         0.903570
50%         1.000000
75%         1.055863
max         2.729420
Name: Item_Visibility_MeanRatio, dtype: float64
count    5681.000000
mean        1.087117
std         0.315011
min         0.568379
25%         0.891310
50%         1.000000
75%         1.068085
max         3.518782
Name: Item_Visibility_MeanRatio, dtype: float64


In [40]:
#Import library:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

#New variable for outlet
train['Outlet'] = le.fit_transform(train['Outlet_Identifier'])
test['Outlet'] = le.fit_transform(test['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
le = LabelEncoder()
for i in var_mod:
    train[i] = le.fit_transform(train[i])
    test[i] = le.fit_transform(test[i])

In [41]:
#Dummy Variables:
train = pd.get_dummies(train, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet'])

train.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Item_Outlet_Sales            float64
Outlet_Years                   int64
Item_Visibility_MeanRatio    float64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Item_Type_Combined_0           uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
Outlet_0                       uint8
Outlet_1                       uint8
O

In [43]:
train.shape

(8523, 35)

In [42]:
#Dummy Variables:
test = pd.get_dummies(test, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet'])

test.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Years                   int64
Item_Visibility_MeanRatio    float64
Item_Fat_Content_0             uint8
Item_Fat_Content_1             uint8
Outlet_Location_Type_0         uint8
Outlet_Location_Type_1         uint8
Outlet_Location_Type_2         uint8
Outlet_Size_0                  uint8
Outlet_Size_1                  uint8
Outlet_Size_2                  uint8
Outlet_Type_0                  uint8
Outlet_Type_1                  uint8
Outlet_Type_2                  uint8
Outlet_Type_3                  uint8
Item_Type_Combined_0           uint8
Item_Type_Combined_1           uint8
Item_Type_Combined_2           uint8
Outlet_0                       uint8
Outlet_1                       uint8
Outlet_2                       uint8
O

In [44]:
test.shape

(5681, 34)

In [45]:
train.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Item_Outlet_Sales,Outlet_Years,Item_Visibility_MeanRatio,...,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.30,0.016047,Dairy,249.8092,OUT049,1999,3735.1380,14,0.922960,...,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,0.019278,Soft Drinks,48.2692,OUT018,2009,443.4228,4,1.003057,...,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.016760,Meat,141.6180,OUT049,1999,2097.2700,14,0.831990,...,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.015274,Fruits and Vegetables,182.0950,OUT010,1998,732.3800,15,1.000000,...,1,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.008082,Household,53.8614,OUT013,1987,994.7052,26,1.000000,...,0,1,0,0,0,0,0,0,0,0


In [46]:
train.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)
test.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)

In [47]:
#Export files as modified versions:
train.to_csv("./data/train_FE.csv",index=False)
test.to_csv("./data/test_FE.csv",index=False)